This notebook is modified from the unbond-generics example on hackage and it repo.

If you have not run `./install.sh` alreay on Jupyter terminal, and running it here for the first time,
`./install.sh` will install dependencies specified in the script into the LTS snapshot directory.
However, on the very first run, it will fail to import modules becuase the running kernel would still be oblivious to the newly installed pakcages.
In such case, you should restart the kernel. Once the kernel is restarted, the `./install.sh` won't do anyting because everything is to date and the newly loaded kernel should be able to see the newly installed package modules.

In [1]:
:!./install.sh

# Similar vs. Bisimilar

Both are concepts that are used for comparing one labeled transision system (LTS) to another.
This notebook demonstrate a well known example that contrasts the two concepts using Haskell.

In a real-world conversation, when we say that ***A*** "simulates" ***B***,
it often means that ***A*** can do most of the things that ***B*** can do but maybe not everything.
For example, it is said that a virtual machine or an emulator software simulates some actual machine,
menaing that it can mimic most of the actual machine but maybe not certain things such as hardware acelleration.

However, in the context of formal system design, when we say that ***A*** *simulates* ***B***, it means that
***A*** must be able to do exactly everything that ***B*** does, and maybe more. When ***B*** also simulates ***A***,
then we say that ***A*** and ***B*** are *similar*. That is, when two systems mutually simulates each other
then they are said to be similar.

## definition of LTS
Let us consider simple LTSs of finite states
defined by a start state and labeled edges, where the the labels represent observable actions.
After an action, it transitions to a next state.
They are pairs of start state and triples, where each triple represents a transtion.
A transitoin is defined by its current state, label, and next state.

We define two LTSs, `systemP` and `systemQ`, in Haskell as below.
Note that we allow some non-determinisim here so that there can be multiple possible next states for the same observable action.
For instance, from the start state `0` of `systemP`, it can transit to either state `2` or `3` after the action labeled by `"a"`.

In [2]:
systemP = ( 0, [(0,"a",1), (0,"a",2), (1,"b",3), (2,"c",4)] ) -- (startState, edges)
systemQ = ( 0, [(0,"a",1), (1,"b",3), (1,"c",4)] )            -- (startState, edges)

We also render these sytems using graphviz DOT format via [Gravizo](http://www.gravizo.com/) for more intuitive visualization.
To make a clear comparision between the systems, a prefix letter is attached (e.g., p2 for state `2` of `systemP` and q2 for state `2` of `systemQ`) when displaying a system as a directed graph.

In [3]:
import IHaskell.Display

displayLTS prefix (s,es) = html $ displayLTSsrc prefix (s,es)
displayLTSsrc prefix (s,es) = gravizo $ digraphDOT prefix (s,es)
gravizo digraphSrc = "<img src='https://g.gravizo.com/svg?"++digraphSrc++"'/>"
digraphDOT prefix (s,es) = "digraph G { "
  ++ concat [ state p++" -> "++state q++" [label="++show a++"]; " | (p,a,q)<-es]
  ++ "start -> "++state s++"; "
  ++ "start [shape=none]; "
  ++ "rankdir = TB; {rank = same; start; "++state s++";} "
  ++"}"
  where state = (prefix++) . show

In [4]:
displayLTS "p" systemP

p1 [label="a"]; p0 -> p2 [label="a"]; p1 -> p3 [label="b"]; p2 -> p4 [label="c"]; start -> p0; start [shape=none]; rankdir = TB; {rank = same; start; p0;} }'/>

In [5]:
displayLTS "q" systemQ

q1 [label="a"]; q1 -> q3 [label="b"]; q1 -> q4 [label="c"]; start -> q0; start [shape=none]; rankdir = TB; {rank = same; start; q0;} }'/>

## definition of simulation
Let $\mathcal{R}$ be a binary relation that relates the states of the two labeled transition systems,
system $Q$ starting from $q_0$ and system $P$ starting from $p_0$.

We say that $Q$ simulates $P$ when the relation $\mathcal{R}$ satisfiing the following two conditions can be defined.
 * $p_0 \mathcal{R}\,q_0$
 * $\forall a, \forall p_i,~ p \xrightarrow{~a~} p_i, ~ \exists q_j,~ q \xrightarrow{~a~} q_j\;$ such that $\;p_i\mathcal{R}\,q_j$

Such a relation $\mathcal{R}$ is called a simulation relation.
We also say that $q_j$ simulates $p_i$ for specific pair of states when $p_i\mathcal{R}\,q_j$.
Because this relation is a preorder, notations such as $p_i \preceq q_j$ is often used for the simulation relation.

Instead of directly building the reation $\mathcal{R}$ as a collection of tuples,
we implement it in Haskell as a binary function (`sim`) with a boolean result,
which corresponds to testing membership (i.e., $(q_j,p_i)\in\mathcal{R}$), as follows.

In [6]:
-- test whether  sysP starting from p0   simulates  sysQ starting from q0
simulates sysQ@(q0,esQ) sysP@(p0,esP) = sim p0 q0
  where
  sim p q = and [or [sim q1 p1 | (q',b,q1) <- esQ, q==q', a==b] -- (or  ...) exists a mathcing step from q
                               | (p',a,p1) <- esP, p==p']       -- (and ...) for each step starting from p

`similar` is defined as mutual simulation.

In [7]:
similar sysP sysQ = simulates sysP sysQ && simulates sysQ sysP

`systemQ` simulates `systemP` but not the other way around.

In [8]:
systemP `simulates` systemQ
systemQ `simulates` systemP
systemP `similar` systemQ

True

False

False

This is because the state $q_1$ in `systemQ` can do anything $p_1$ can do and also do anything $p_2$ can do in `systemP`.

However, neither the state $p_1$ nor the state $p_2$ in `systemP` can do everything $q_1$ does in `systemQ`.
Observe that $p_1$ cannot follow the step with label $c$ and that $p_2$ cannot follow the step with label $b$.

## mutual simulation
Add one more transition $p_2 \xrightarrow{~b~} p_3$ to `systemP`, and let us call it `systemP'`.

In [9]:
systemP' = (0, [(0,"a",1), (0,"a",2), (1,"b",3), (2,"b",3), (2,"c",4)]) -- (startState, edges)

displayLTS "p" systemP'

p1 [label="a"]; p0 -> p2 [label="a"]; p1 -> p3 [label="b"]; p2 -> p3 [label="b"]; p2 -> p4 [label="c"]; start -> p0; start [shape=none]; rankdir = TB; {rank = same; start; p0;} }'/>

Observe that `systemP'` and `systemQ` are similar. That is, `systemP'` can also simulate `systemQ` as well as being simulated by `systemQ`.

In [10]:
systemP' `simulates` systemQ
systemQ  `simulates` systemP'
systemP' `similar` systemQ

True

False

False

## definition of bisimulation
Let $\mathcal{R}$ be a binary relation that relates the states of the two labeled transition systems,
system $P$ starting from $p_0$ and system $Q$ starting from $q_0$.

We say that $P$ bisimulates $Q$, or $P$ and $Q$ are bisimilar, when the relation $\mathcal{R}$ satisfiing the following three conditions can be defined.
 * $p_0 \mathcal{R}\,q_0$
 * $\forall a, \forall p_i,~ p \xrightarrow{~a~} p_i, ~ \exists q_j,~ q \xrightarrow{~a~} q_j\;$ such that $\;p_i\mathcal{R}\,q_j$
 * $\forall a, \forall q_j,~ q \xrightarrow{~a~} q_j, ~ \exists p_i,~ p \xrightarrow{~a~} p_i\;$ such that $\;p_i\mathcal{R}\,q_j$ 

Such a relation $\mathcal{R}$ is called a bisimulation relation.
We also say that $p_i$ and $p_j$ are bisimilar for specific pair of states when $p_i\mathcal{R}\,q_j$.
Because this relation is an equivalence, the notation $p_i \sim q_j$ is often used for the simulation relation.

Instead of directly building the reation $\mathcal{R}$ as a collection of tuples,
we implement it in Haskell as a binary function (`bisim`) with a boolean result,
which corresponds to testing membership (i.e., $(p_i,q_j)\in\mathcal{R}$), as follows.

In [11]:
-- test whether  sysP starting from p0   is bisimilar to  sysQ starting from q0
bisimilar sysP@(p0,esP) sysQ@(q0,esQ) = bisim p0 q0
  where
  bisim p q = and [or [bisim p1 q1 | (q',b,q1) <- esQ, q==q', a==b]
                                   | (p',a,p1) <- esP, p==p']
           && and [or [bisim p1 q1 | (p',a,p1) <- esQ, p==p', b==a]
                                   | (q',b,q1) <- esP, q==q']

In [12]:
similar    systemP  systemQ
bisimilar  systemP  systemQ -- obviously false, was not even simialr

False

False

In [13]:
similar    systemP' systemQ
bisimilar  systemP' systemQ -- oh! similar but not bisimilar ?!?!

False

False

### so, what happned?

Here is where bisimulation fails.
 1. Let `systemP'` take the first leading step $p_0 \xrightarrow{~a~} p_1$.
      - Then, there is no othre choice for `systemQ` than following by $q_0 \xrightarrow{~a~} q_1$.
 1. This time, let `systemQ` take the second leading step $q_1 \xrightarrow{~c~} q_4$. 
      - Then, `systemP` **cannot** follow this step because there is no transition with label $c$ from $p_1$.
      
When testing similarity with the function `similar`, above bisimulation steps are not considered because the roles of leader and follower are fixed in each of the two simulation tests. One system takes the leading role and the other takes the following role throughut a simulation test. In contrast, both roles are considered for both systems in every step of the bisimulation. This is why bisimulation is a stronger (or, more fine-grained) notion of equivalence than mutual simulation.